In [1]:
#%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import marshaltools
from ast import literal_eval
import logging
from astropy.time import Time
from ipywidgets import interactive
import ipywidgets as widgets
import re, os
import sncosmo
import json

from utils import get_config
from utils import has_iband


In [2]:
# If you are running this notebook for the first time, you will be prompted to enter your username and
# the path to your downloads from slack. If you entered something incorrectly, you can correct this in .config
config = get_config()

username = config['username']
date = '2018-09-29'
maxz = 0.1
minpeakmag = 19.5   # A candidate need to have at least one detection brigther than this
mindet = 5          # A candidate need to have at least this many detections
maxage = 40         # If a detection has an age older than this, skip (stars,age). 
                    # Q: How can histories be older than 30days?
                    # At this stage we will also cut history older than this in fit
SNcut = 5           # Need at least some photometry with this Signal to Noise
minrefframes = 1    # Cut away alert information where this is not positive (how can it not be, but ok...)
lastobsage = 5      # Do not display objects for which the last obs is older than this


marshal_savid = 42
logpath = 'logs/'

# Get the other set of marshal source ids. Lets not talk about, took too much of my life alrady
with open('sne_following.json', 'r') as fp:
    sne_following = json.load(fp)

# log
logger = logging.getLogger(username)
handler = logging.FileHandler(os.path.join(logpath, 'ztfcosmo_trigger_%s_%s.log'%(date,username)))
handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)

manual_inspection=[]

In [3]:
# Now fun starts for real. Also load cosmology list to make sure we are not missing types from there
pl = marshaltools.ProgramList('AMPEL Test',load_candidates=False)
cpl = marshaltools.ProgramList('Cosmology',load_candidates=False)


INFO:marshaltools.ProgramList:Initialized ProgramList for program AMPEL Test (ID 4)
INFO:marshaltools.ProgramList:Loaded 213 saved sources for program AMPEL Test.
INFO:marshaltools.ProgramList:Initialized ProgramList for program Cosmology (ID 2)
INFO:marshaltools.ProgramList:Loaded 454 saved sources for program Cosmology.


In [4]:
df_phot = pd.read_csv(os.path.join(config['slackfilepath'], "Photometry_%s.csv"%(date)))
if minrefframes>0:
    df_phot = df_phot[df_phot["nframesref"]>minrefframes]
    logger.info("Cutting down alert info to subset with nrefframes keyword")

INFO:jnordin:Cutting down alert info to subset with nrefframes keyword


In [5]:
# This is the timeconsuming part. Downloading all the data and do saltfits. Let it start and get a coffee
for count, sn in enumerate(pl.sources.keys()):
    if not pl.sources[sn]['classification'] is None:
        logger.info("SN %s classified with type %s, skipping."%(sn,pl.sources[sn]['classification']))
        continue
    if sn in cpl.sources.keys() and not cpl.sources[sn]['classification'] is None:
        logger.info("SN %s classified with type %s in Cosmology, skipping."%(sn,cpl.sources[sn]['classification']))
        continue
    

    if 'SNCOSMOFIT' in pl.sources[sn]:
        print('already done')
        continue
        
    # Download specific LC
    lc = pl.get_lightcurve(sn)
    del pl.lightcurves[sn]

    # Check nbr filters filters
    if len(np.unique(lc.table["filter"][lc.table["magpsf"]<99] ))<2:
        logger.info("SN %s has too few filters for saltfit."%(sn))
        continue
    
    
    # create a model
    model = sncosmo.Model(source='salt2')
    
    # Massage data
    data = lc.table_sncosmo
    data["mjd"] = data["mjd"]-Time.now().mjd
    data = data[ (data["mjd"]>-maxage) ]
    
    # Replicating data quality checks
    signalNoise = np.abs(data["flux"])/data["fluxerr"]
    data = data[ (signalNoise>SNcut) ]
    
    
    
    # run the fit
    if len(signalNoise[signalNoise>SNcut])==0:
        manual_inspection.append(sn)
        logger.info("Not enugh good data for SN %s. Manual inspection prob needed."%(sn))
        continue    
 
    if np.max(data["mjd"])<-lastobsage:
        logger.info("SN %s had last obs %s days go, skipping."%(sn,np.max(data["mjd"])))
        continue



    try:
        result, fitted_model = sncosmo.fit_lc(
            data, model,
            ['z', 't0', 'x0', 'x1', 'c'],  # parameters of model to vary
            bounds={'z':(0., 0.2),'x1':(-5.,5.),'t0':(-20.,15.),'c':(-1.,10.)})  # bounds on parameters (if any)
        pl.sources[sn]['SNCOSMOfit'] = result, fitted_model, data
#        sncosmo.plot_lc(data, model=fitted_model, errors=result.errors)
#        plt.show()

            
    except ValueError:
        print("Why does fit fail for %s?"%(sn))
        manual_inspection.append(sn)
        logger.info("SALT fit fails for SN %s. Manual inspection prob needed."%(sn))
        continue

        
        

INFO:jnordin:SN ZTF18aakgsmi had last obs -6.6480160854625865 days go, skipping.
INFO:jnordin:SN ZTF18aasdted classified with type SN Ia, skipping.
INFO:jnordin:SN ZTF18aauvptt classified with type AGN, skipping.
INFO:jnordin:SN ZTF18abmwzws had last obs -16.650724694096425 days go, skipping.
INFO:jnordin:SN ZTF18aawlhkh classified with type AGN, skipping.
INFO:jnordin:SN ZTF18abddhaa had last obs -12.602442462564795 days go, skipping.
INFO:jnordin:SN ZTF18abgucjb had last obs -11.413845602313813 days go, skipping.
INFO:jnordin:SN ZTF18ablvpuj had last obs -16.64146792019892 days go, skipping.
INFO:jnordin:SN ZTF18ablqqeb had last obs -18.611489111739502 days go, skipping.
INFO:jnordin:Not enugh good data for SN ZTF18ablviog. Manual inspection prob needed.
INFO:jnordin:SN ZTF18ablwwwb had last obs -20.647526439141075 days go, skipping.
INFO:jnordin:SN ZTF18ablowct had last obs -17.50210701017204 days go, skipping.
INFO:jnordin:SN ZTF18ablprcf classified with type AGN, skipping.
INFO:jn

INFO:jnordin:SN ZTF18abshezu classified with type SLSN-I, skipping.
INFO:jnordin:SN ZTF18abryewg classified with type SN Ia 91T-like, skipping.
INFO:jnordin:SN ZTF18abssukb classified with type SN Ia, skipping.
INFO:jnordin:SN ZTF18abscygs classified with type SN Ia, skipping.
INFO:jnordin:SN ZTF18absdjms classified with type SN Ia, skipping.
INFO:jnordin:SN ZTF18ablhsgt had last obs -25.646490191895282 days go, skipping.
INFO:jnordin:SN ZTF18abnorzb had last obs -13.641410492164141 days go, skipping.
INFO:jnordin:SN ZTF18abottbj had last obs -13.39882947467413 days go, skipping.
INFO:jnordin:SN ZTF18absqsey had last obs -17.30760765557352 days go, skipping.
INFO:jnordin:SN ZTF18abufaej classified with type SN II, skipping.
INFO:jnordin:SN ZTF18abtogdl classified with type SN Ia, skipping.
INFO:jnordin:SN ZTF18absumwc has too few filters for saltfit.
INFO:jnordin:SN ZTF18absmqmw had last obs -20.627401166930213 days go, skipping.
INFO:jnordin:SN ZTF18abmwxvv had last obs -24.5943268780

In [6]:
visual_inspect = []
iband_sne = []
for sn in pl.sources.keys():
    if not 'SNCOSMOfit' in pl.sources[sn].keys():
        continue
        
    # Look for Iband coverage
    fields = np.unique( df_phot['field'][ df_phot["ztf_name"]==sn ] )
    if len(fields)>0 and has_iband(fields,['September','October']):
        iband_sne.append(sn)      
    
    visual_inspect.append(sn)
        
        
#    visual_inspect.append(sn)
print("Found %s lightcurves  to inspect"%(len(visual_inspect)))     
print("... out of these %s have iband"%(len(iband_sne)))     



Found 58 lightcurves  to inspect
... out of these 23 have iband


In [7]:
print(visual_inspect)

['ZTF18aaupjtd', 'ZTF18abddgzz', 'ZTF18ablllyw', 'ZTF18aayhpph', 'ZTF17aaaafcd', 'ZTF18abcqvhs', 'ZTF18abnujuy', 'ZTF18abnnnzk', 'ZTF18abporae', 'ZTF18abpudlt', 'ZTF18abrwswl', 'ZTF18abrzuqw', 'ZTF18ablmduj', 'ZTF18absloer', 'ZTF18abtkctg', 'ZTF18abklshn', 'ZTF18abthaii', 'ZTF18absnsqx', 'ZTF18abrwrch', 'ZTF18abuicad', 'ZTF18absbspk', 'ZTF18abuluko', 'ZTF18abtghbi', 'ZTF18abtswjk', 'ZTF18aburkuo', 'ZTF18abuksxd', 'ZTF18abvrzqj', 'ZTF18abtxvhy', 'ZTF18abvnqqk', 'ZTF18abwbtco', 'ZTF18abvxqtp', 'ZTF18abuioue', 'ZTF18abvvqtf', 'ZTF18abwntpz', 'ZTF18abwqgsc', 'ZTF18abwktmm', 'ZTF18abucflq', 'ZTF18abxyudf', 'ZTF18abvrgjc', 'ZTF18abxwiqz', 'ZTF18abxbhov', 'ZTF18abwkxgo', 'ZTF18abwlejr', 'ZTF18abxdkny', 'ZTF18abwlpzf', 'ZTF18abqdwze', 'ZTF18abxumzx', 'ZTF18abvfecb', 'ZTF18abxdkni', 'ZTF18abwzedh', 'ZTF18absmogv', 'ZTF17aaaadqv', 'ZTF18abwfmot', 'ZTF18abvnwzz', 'ZTF18absgvqb', 'ZTF17aabihdn', 'ZTF18abmmszc', 'ZTF18abbmenz']


In [8]:
if "ZTF18abscygs" in visual_inspect:
    print("bu")

In [9]:
# Function to make stupid plot of a SN lightcurve
def plotty(i):
    """
    Flash SNCOSMO fit plot
    """
    
    global snnbr
    
    
    # Evaluate reply
    snname = visual_inspect[snnbr]
    if i == 'TooLate':
        logger.info("%s Faded too much while waiting. Call P200 or Keck?"%(snname))
        decisions[snnbr] = i
        snnbr += 1
    elif i == 'VeryGood':
        logger.info("Nice looking likely SN %s."%(snname))
        decisions[snnbr] = i
        snnbr += 1
    elif i == 'Possibly':
        logger.info("Get SN %s if we can."%(snname))
        decisions[snnbr] = i
        snnbr += 1    
    elif i == 'NoSNIa':
        logger.info("Object %s most likely not SNIa. "%(snname))
        decisions[snnbr] = i
        snnbr += 1
    elif i == 'BadData':
        logger.info("Too little data of %s, skip. "%(snname))
        decisions[snnbr] = i
        snnbr += 1    
    elif i == 'GoBack':
        logger.info("I want to scan some more, go back!")
        snnbr -= 1
    elif i=='Nothing':
        # Lets do nothing
        pass
    
    # Are we done?
    if snnbr==len(visual_inspect):
        print( "Seems like we are all done. How do we exit?")
        return False
    
    
    
    # Reset    
    snname = visual_inspect[snnbr]
    
    # Retrieve saltfit
    result, fitted_model, data = pl.sources[snname]['SNCOSMOfit']  
    sncosmo.plot_lc(data, model=fitted_model, errors=result.errors, 
                    figtext='%s (%i/%i)'%(snname,snnbr+1,len(visual_inspect)))
    plt.show()
    wiggy.value = 'Nothing'
    
    return (i)


wiggy = widgets.RadioButtons(
    options=['Nothing','TooLate', 'VeryGood', 'Possibly','NoSNIa','BadData','GoBack'],
    value='Nothing',
    description='Action:',
    disabled=False
)
y = interactive(plotty,i = wiggy)

In [10]:
# Presumably you want to scan from the first SN, but you can change this and run display again to revisit something
# You cant jump ahead in the list though
snnbr = 0
# This is the all imuportant list of decisions you have made
decisions = {}

In [11]:
# This is the scanning box! 
# You have three choices (Nothing is not a choice and GoBack steps lets you go back in order)
# - Wait : Select this if the transient is rizing and might get into RCF range (~<18.7)
# - Submit : Should probably get a spectrum (you do not need to worry about where just yet)
# - Garbage : Variable star or clearly non SNIa. Evanetually these will be rejected and never more show up
display(y)

interactive(children=(RadioButtons(description='Action:', options=('Nothing', 'TooLate', 'VeryGood', 'Possibly…

In [12]:
# Go through current list of SNe on the obs queue. If already 
for snnbr, choice in decisions.items():
    if not (choice=='VeryGood' or choice=='Possibly'):
        continue
    sn = visual_inspect[snnbr]
    print("%s %s"%(sn,choice))

    
    if sn in sne_following.keys():
        logger.info("%s already in obs queue. Adding with setting %s."%(sn,choice))
        sne_following[sn].append([choice,username,date])
    else:
        logger.info("Adding %s to obs queue with setting %s."%(sn,choice))
        sne_following[sn] = [ [choice,username,date] ]

    if sn in iband_sne:
        sne_following[sn][-1].append("I-band")
        logger.info("...with I-band!")

        

INFO:jnordin:ZTF18ablllyw already in obs queue. Adding with setting Possibly.
INFO:jnordin:...with I-band!
INFO:jnordin:ZTF18abrwswl already in obs queue. Adding with setting VeryGood.
INFO:jnordin:ZTF18abtkctg already in obs queue. Adding with setting VeryGood.
INFO:jnordin:...with I-band!
INFO:jnordin:ZTF18abthaii already in obs queue. Adding with setting Possibly.
INFO:jnordin:ZTF18absnsqx already in obs queue. Adding with setting VeryGood.
INFO:jnordin:...with I-band!
INFO:jnordin:ZTF18abrwrch already in obs queue. Adding with setting VeryGood.
INFO:jnordin:...with I-band!
INFO:jnordin:ZTF18abuicad already in obs queue. Adding with setting VeryGood.
INFO:jnordin:ZTF18absbspk already in obs queue. Adding with setting Possibly.
INFO:jnordin:ZTF18abuluko already in obs queue. Adding with setting Possibly.
INFO:jnordin:...with I-band!
INFO:jnordin:ZTF18abtghbi already in obs queue. Adding with setting VeryGood.
INFO:jnordin:ZTF18abtswjk already in obs queue. Adding with setting Possibl

ZTF18ablllyw Possibly
ZTF18abrwswl VeryGood
ZTF18abtkctg VeryGood
ZTF18abthaii Possibly
ZTF18absnsqx VeryGood
ZTF18abrwrch VeryGood
ZTF18abuicad VeryGood
ZTF18absbspk Possibly
ZTF18abuluko Possibly
ZTF18abtghbi VeryGood
ZTF18abtswjk Possibly
ZTF18aburkuo VeryGood
ZTF18abuksxd Possibly
ZTF18abvrzqj VeryGood
ZTF18abtxvhy Possibly
ZTF18abvnqqk VeryGood
ZTF18abwbtco Possibly
ZTF18abvxqtp Possibly
ZTF18abuioue VeryGood
ZTF18abvvqtf Possibly
ZTF18abwqgsc Possibly
ZTF18abwktmm VeryGood
ZTF18abucflq VeryGood
ZTF18abxbhov Possibly
ZTF18abwkxgo Possibly
ZTF18abwlejr Possibly
ZTF18abxdkny Possibly
ZTF18abwlpzf Possibly
ZTF18abxdkni Possibly
ZTF18abwzedh Possibly
ZTF18absgvqb VeryGood


In [13]:
# Unfortunately not done yet. First check whether any of our SNe to obs are bright enough for RCF.
for snname in sne_following.keys():
    
    lc = pl.get_lightcurve(snname)
    del pl.lightcurves[snname]
    peakmag = np.min(lc.table['magpsf'])
    recentmag =  np.mean(lc.table['magpsf'][ (lc.table['jdobs']-Time.now().jd)>-5 ] )
    
    lastchoice = sne_following[snname][-1][0]
    
    if peakmag<18.9 and recentmag < 20:
        logger.info("%s could be RCF target? peakmag %s recentmag %s."%(sn,peakmag,recentmag))
        msg = 'http://skipper.caltech.edu:8080/cgi-bin/growth/view_source.cgi?name=%s'%(snname)
        print( "Follow link and trigger (RCF) SEDM obs. Our prio was %s"%(lastchoice) )
        print(msg)
        
    # Have disabled the Cosmology SEDM triggers as we have no time currently
    #elif peakmag<19.5 and recentmag < 20:
    #    logger.info("%s could be Cosmology SEDM target? peakmag %s recentmag %s."%(sn,peakmag,recentmag))
    #    msg = 'http://skipper.caltech.edu:8080/cgi-bin/growth/view_source.cgi?name=%s'%(snname)
    #    print( "Follow link and trigger (Cosmologz) SEDM obs with prio %s"%(lastchoice) )
    #    print(msg)
    


INFO:jnordin:ZTF18absgvqb could be RCF target? peakmag 18.35 recentmag 19.86.


Follow link and trigger (RCF) SEDM obs. Our prio was Possibly
http://skipper.caltech.edu:8080/cgi-bin/growth/view_source.cgi?name=ZTF18abcqvhs


/home/jnordin/local/miniconda2/envs/py36/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2954: RuntimeWarning: Mean of empty slice.
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/home/jnordin/local/miniconda2/envs/py36/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
INFO:jnordin:ZTF18absgvqb could be RCF target? peakmag 18.21 recentmag 19.017999999999997.


Follow link and trigger (RCF) SEDM obs. Our prio was VeryGood
http://skipper.caltech.edu:8080/cgi-bin/growth/view_source.cgi?name=ZTF18abscygs


INFO:jnordin:ZTF18absgvqb could be RCF target? peakmag 18.76 recentmag 19.084285714285716.


...

Follow link and trigger (RCF) SEDM obs. Our prio was Possibly
http://skipper.caltech.edu:8080/cgi-bin/growth/view_source.cgi?name=ZTF18abxbhov


INFO:jnordin:ZTF18absgvqb could be RCF target? peakmag 18.88 recentmag 19.05625.


Follow link and trigger (RCF) SEDM obs. Our prio was VeryGood
http://skipper.caltech.edu:8080/cgi-bin/growth/view_source.cgi?name=ZTF18abvtrnc


INFO:jnordin:ZTF18absgvqb could be RCF target? peakmag 18.43 recentmag 18.476666666666667.


Follow link and trigger (RCF) SEDM obs. Our prio was Possibly
http://skipper.caltech.edu:8080/cgi-bin/growth/view_source.cgi?name=ZTF18abvxqtp


In [ ]:
# Now we have to go back to the sne selected for manual verification below:
# Trigger observations of these and add name below
for sn in manual_inspection:
    msg = 'http://skipper.caltech.edu:8080/cgi-bin/growth/view_source.cgi?name=%s'%(sn)
    print( "Suggested have a manual look at %s"%(sn) )
    print(msg)
    

In [22]:
# Save the list of SN w triggered obs
with open('sne_following.json', 'w') as fp:
    json.dump(sne_following, fp)


In [ ]:
# Enter the names of the SNe you triggered SEDM obs for
sedmtrig = []
logger.info("Triggered SEDM obs of: %s"%(sedmtrig))

In [14]:
# Check how many of the followed SNe now have types:
for snname in sne_following.keys():
     if not pl.sources[snname]['classification'] is None:
        print("%s has been typed as %s!"%(snname,pl.sources[snname]['classification']))


ZTF18abscygs has been typed as SN Ia!
ZTF18abtogdl has been typed as SN Ia!
ZTF18abwtops has been typed as SN Ia!
ZTF18abxygvv has been typed as SN Ia!
ZTF18abwnsoc has been typed as SN Ia!
ZTF18absoomk has been typed as SN Ia!
ZTF18abvtrnc has been typed as SN Ia!


In [7]:
scandec = {0: 'Garbage', 1: 'Wait', 2: 'Garbage', 3: 'Garbage', 4: 'Garbage', 5: 'Submit', 6: 'Submit', 7: 'Wait', 8: 'Submit', 9: 'Garbage', 10: 'Garbage', 11: 'Wait', 12: 'Wait', 13: 'Garbage', 14: 'Garbage', 15: 'Garbage', 16: 'Garbage', 17: 'Submit', 18: 'Garbage', 19: 'Garbage', 20: 'Garbage', 21: 'Garbage', 22: 'Garbage', 23: 'Wait', 24: 'Wait', 25: 'Garbage', 26: 'Garbage', 27: 'Garbage', 28: 'Garbage', 29: 'Submit', 30: 'Garbage', 31: 'Garbage', 32: 'Wait', 33: 'Garbage', 34: 'Garbage', 35: 'Garbage', 36: 'Garbage', 37: 'Garbage', 38: 'Garbage', 39: 'Garbage', 40: 'Wait', 41: 'Submit', 42: 'Submit', 43: 'Submit', 44: 'Garbage', 45: 'Submit', 46: 'Garbage', 47: 'Wait', 48: 'Garbage', 49: 'Garbage'}


In [17]:
print(pl.sources["ZTF18abscygs"])

{'rcid': 37, 'creationdate': '2018-09-04', 'name': 'ZTF18abscygs', 'classification': 'SN Ia', 'redshift': '0.03', 'iauname': None, 'release_status': 'pending', 'field': 743, 'candid': 611487213715015101, 'ra': 73.5278345, 'lastmodified': '2018-09-27', 'release_auth': None, 'dec': 48.9004181, 'id': 4416, 'fields': array([ 743, 1781]), 'ccds': array([9, 1])}


In [18]:
print(pl.sources["ZTF18abtogdl"])

{'rcid': 27, 'creationdate': '2018-09-08', 'name': 'ZTF18abtogdl', 'classification': 'SN Ia', 'redshift': '0.105', 'iauname': None, 'release_status': 'pending', 'field': 797, 'candid': 615259722715015081, 'ra': 267.7971078, 'lastmodified': '2018-09-27', 'release_auth': None, 'dec': 53.3144701, 'id': 4671, 'fields': array([797]), 'ccds': array([6])}
